In [23]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [24]:
data = pd.read_csv('data/apple_stock.csv')

In [25]:
data.head()

,Date,Adj Close,Close,High,Low,Open,Volume
0,03-01-2025,243.860001,243.860001,244.179993,241.889999,243.369995,15135053
1,02-01-2025,243.850006,243.850006,249.100006,241.820007,248.929993,55558000
2,31-12-2024,250.419998,250.419998,253.279999,249.429993,252.440002,39480700
3,30-12-2024,252.199997,252.199997,253.500000,250.750000,252.229996,35557500
4,27-12-2024,255.589996,255.589996,258.700012,253.059998,257.829987,42355300


In [26]:
data['Date'] = pd.to_datetime(data['Date'], dayfirst=True)
data.set_index('Date', inplace=True)
data = data[['Close']]

In [27]:
scaler = MinMaxScaler(feature_range=(0, 1))
data['Close'] = scaler.fit_transform(data[['Close']])

In [28]:
def create_sequences(data, seq_length=60):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

seq_length = 60
X, y = create_sequences(data['Close'].values, seq_length)

In [29]:
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

In [30]:
lstm_model = Sequential()
lstm_model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
lstm_model.add(LSTM(units=50))
lstm_model.add(Dense(1))

e:\PROJECTS\Hybrid_ML_model\venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [31]:
lstm_model.compile(optimizer='adam', loss='mean_squared_error')
lstm_model.fit(X_train, y_train, epochs=20, batch_size=32)

Epoch 1/20
277/277 ━━━━━━━━━━━━━━━━━━━━ 35s 66ms/step - loss: 0.0044
Epoch 2/20
277/277 ━━━━━━━━━━━━━━━━━━━━ 15s 52ms/step - loss: 1.0207e-04
Epoch 3/20
277/277 ━━━━━━━━━━━━━━━━━━━━ 13s 46ms/step - loss: 9.7249e-05
Epoch 4/20
277/277 ━━━━━━━━━━━━━━━━━━━━ 13s 47ms/step - loss: 6.1468e-05
Epoch 5/20
277/277 ━━━━━━━━━━━━━━━━━━━━ 13s 45ms/step - loss: 6.6089e-05
Epoch 6/20
277/277 ━━━━━━━━━━━━━━━━━━━━ 16s 59ms/step - loss: 4.8281e-05
Epoch 7/20
277/277 ━━━━━━━━━━━━━━━━━━━━ 14s 49ms/step - loss: 5.3913e-05
Epoch 8/20
277/277 ━━━━━━━━━━━━━━━━━━━━ 16s 59ms/step - loss: 8.7142e-05
Epoch 9/20
277/277 ━━━━━━━━━━━━━━━━━━━━ 13s 46ms/step - loss: 5.5519e-05
Epoch 10/20
277/277 ━━━━━━━━━━━━━━━━━━━━ 13s 47ms/step - loss: 4.8333e-05
Epoch 11/20
277/277 ━━━━━━━━━━━━━━━━━━━━ 13s 48ms/step - loss: 4.5052e-05
Epoch 12/20
277/277 ━━━━━━━━━━━━━━━━━━━━ 14s 49ms/step - loss: 3.7165e-05
Epoch 13/20
277/277 ━━━━━━━━━━━━━━━━━━━━ 11s 38ms/step - loss: 3.9679e-05
Epoch 14/20
277/277 ━━━━━━━━━━━━━━━━━━━━ 11s 38ms/s

In [32]:
data['Lag_1'] = data['Close'].shift(1)
data['Lag_2'] = data['Close'].shift(2)
data['Lag_3'] = data['Close'].shift(3)
data = data.dropna()

In [33]:
X_lin = data[['Lag_1', 'Lag_2', 'Lag_3']]
y_lin = data['Close']
X_train_lin, X_test_lin = X_lin[:train_size], X_lin[train_size:]
y_train_lin, y_test_lin = y_lin[:train_size], y_lin[train_size:]

In [34]:
lin_model = LinearRegression()
lin_model.fit(X_train_lin, y_train_lin)

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [ ]:
X_test_lstm = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
lstm_predictions = lstm_model.predict(X_test_lstm)
lstm_predictions = scaler.inverse_transform(lstm_predictions)

70/70 ━━━━━━━━━━━━━━━━━━━━ 4s 29ms/step


In [ ]:
lin_predictions = lin_model.predict(X_test_lin)
lin_predictions = scaler.inverse_transform(lin_predictions.reshape(-1, 1))

In [35]:
min_len = min(len(lstm_predictions), len(lin_predictions))
lstm_predictions = lstm_predictions[:min_len]
lin_predictions = lin_predictions[:min_len]

In [36]:
hybrid_predictions = (0.7 * lstm_predictions) + (0.3 * lin_predictions)

In [37]:
lstm_future_predictions = []
last_sequence = X[-1].reshape(1, seq_length, 1)
for _ in range(10):
    lstm_pred = lstm_model.predict(last_sequence)[0, 0]
    lstm_future_predictions.append(lstm_pred)
    lstm_pred_reshaped = np.array([[lstm_pred]]).reshape(1, 1, 1)
    last_sequence = np.append(last_sequence[:, 1:, :], lstm_pred_reshaped, axis=1)
lstm_future_predictions = scaler.inverse_transform(np.array(lstm_future_predictions).reshape(-1, 1))

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 209ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step


In [40]:
recent_data = data['Close'].values[-3:]
lin_future_predictions = []
for _ in range(10):
    lin_pred = lin_model.predict(recent_data.reshape(1, -1))[0]
    lin_future_predictions.append(lin_pred)
    recent_data = np.append(recent_data[1:], lin_pred)
lin_future_predictions = scaler.inverse_transform(np.array(lin_future_predictions).reshape(-1, 1))

e:\PROJECTS\Hybrid_ML_model\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
e:\PROJECTS\Hybrid_ML_model\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
e:\PROJECTS\Hybrid_ML_model\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
e:\PROJECTS\Hybrid_ML_model\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
e:\PROJECTS\Hybrid_ML_model\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  

In [39]:
hybrid_future_predictions = (0.7 * lstm_future_predictions) + (0.3 * lin_future_predictions)

In [ ]:
future_dates = pd.date_range(start=data.index[-1] + pd.Timedelta(days=1), periods=10)
predictions_df = pd.DataFrame({
    'Date': future_dates,
    'LSTM Predictions': lstm_future_predictions.flatten(),
    'Linear Regression Predictions': lin_future_predictions.flatten(),
    'Hybrid Model Predictions': hybrid_future_predictions.flatten()
})
print(predictions_df)